In [14]:
import pandas as pd

def unique_list(x):
    return sorted({v for v in x if pd.notna(v)})

In [48]:
df_routes=pd.read_parquet(r"dados\routes_historico.parquet")
df_trips=pd.read_parquet(r'dados\trips_historico.parquet')
df_stops=pd.read_parquet(r'dados\stops_historico_categorizado.parquet')
df_frequencies=pd.read_parquet(r"dados\frequencies_historico.parquet")

Informações da tabela de rotas

In [ ]:
df_routes_categorizado= (
    df_routes
    .groupby("route_id")
    .agg(
        route_long_name=("route_long_name", unique_list),
        route_color=("route_color", lambda x: unique_list(x)[0] if len(unique_list(x))>0 else None),
        route_text_color=("route_text_color", lambda x: unique_list(x)[0] if len(unique_list(x))>0 else None),
        datas_operacao=("data_referencia", unique_list)
    )
    .reset_index()
)

Informações da tabela de trips

In [ ]:
df_trips_agrupado = (
    df_trips
    .groupby("route_id")
    .agg(
        direcoes=("direction_id", unique_list),
        service_ids=("service_id", unique_list),    )
    .reset_index()
)
df_trips_agrupado['direcoes']=df_trips_agrupado['direcoes'].apply(lambda x: "Circular" if len(x) == 1 else "Não circular")

df_routes_categorizado=df_routes_categorizado.merge(df_trips_agrupado,on='route_id')

Informações da tabela de frequências

In [ ]:
# Ordena para garantir sequência correta
df_frequencies = df_frequencies.sort_values(['trip_id', 'data_referencia', 'start_time'])

# Função para agrupar blocos
def agrupar_blocos(grupo):
    # Identifica onde começa um novo bloco
    novo_bloco = (
        (grupo['headway_secs'] != grupo['headway_secs'].shift()) |
        (grupo['start_time'] != grupo['end_time'].shift())
    )

    # Cria um ID de bloco cumulativo
    bloco_id = novo_bloco.cumsum()

    blocos_resumo = []
    for _, bloco in grupo.groupby(bloco_id):
        inicio = bloco['start_time'].iloc[0]
        fim = bloco['end_time'].iloc[-1]
        headway_min = bloco['headway_secs'].iloc[0] // 60
        blocos_resumo.append(f"{inicio}–{fim} a cada {headway_min} minutos")

    return blocos_resumo

# Função para determinar se a trip é noturna ou diurna
def periodo_operacao(grupo):
    # Converte horários para minutos desde 00h
    to_min = lambda t: int(t.split(':')[0]) * 60 + int(t.split(':')[1])
    start_mins = grupo['start_time'].map(to_min)
    end_mins = grupo['end_time'].map(to_min)

    if any((start_mins < 280) | (end_mins < 280)):  # antes de 04h40 (280 min)
        return "Noturno"
    return "Diurno"

# Aplica agrupamento
df_resumo = (
    df_frequencies.groupby(['trip_id', 'data_referencia'])
    .apply(lambda g: pd.Series({
        'faixas_horarias': agrupar_blocos(g),
        'Periodo': periodo_operacao(g)
    }))
    .reset_index()
)

print(df_resumo.head())


In [ ]:
df_resumo

,trip_id,data_referencia,faixas_horarias
0,1012-10-0,201608,"[03:00:00–03:59:00 a cada 1800 segundos, 04:00..."
1,1012-10-0,201609,"[03:00:00–03:59:00 a cada 1800 segundos, 04:00..."
2,1012-10-0,201610,"[03:00:00–03:59:00 a cada 1800 segundos, 04:00..."
3,1012-10-0,201611,"[03:00:00–03:59:00 a cada 3600 segundos, 04:00..."
4,1012-10-0,201612,"[03:00:00–03:59:00 a cada 1800 segundos, 04:00..."
...,...,...,...
276255,N843-11-0,202411,"[00:00:00–00:59:00 a cada 1800 segundos, 01:00..."
276256,N843-11-0,202412,"[00:00:00–00:59:00 a cada 1800 segundos, 01:00..."
276257,N843-11-0,202501,"[00:00:00–00:59:00 a cada 1800 segundos, 01:00..."
276258,N843-11-0,202502,"[00:00:00–00:59:00 a cada 1800 segundos, 01:00..."
